In [10]:


import numpy as np
def feature_sign(A, y, g):
    #Step 1
    #dim_A = A.shape[0]
    active_set = set()
    theta = np.zeros(A.shape[1])
    x = np.zeros(y.shape[0])
    
    #Step 2
    term1 = np.dot(A.T, y)
    term2 = np.dot(np.dot(A.T, A), x)
    grad = -2 * (term1 - term2)
    max_grad = np.argmax(np.abs(grad))
    cost_functn = np.dot(y.T,y)
    opt_soln = 0
    tol = 1e-10
    z_opt =np.inf
    while z_opt > g or not np.allclose(opt_soln, 0):
        if np.allclose(opt_soln,0):
             index = np.argmax(np.abs(grad) * (theta == 0))
             if grad[index] > g:
                 theta[index] = -1
                 x[index]=0
                 active_set.add(index)
             elif grad[index] < -g:
                theta[index] = 1
                x[index]=0
                active_set.add(index)
           
             if len(active_set) ==0:
                 break
                
        #3.Feature-sign step
        sub_dim = np.array(sorted(active_set))
        #sub_dim = active_set.sort()
        temp = np.dot(A.T, A)
        A_cap = temp[np.ix_(sub_dim, sub_dim)]
        term1_cap = term1[sub_dim]
        theta_cap = theta[sub_dim]
        b = term1_cap - g * theta_cap/2
        
        
        x_new = np.linalg.solve(A_cap, b)
        new_theta = np.sign(x_new)
        x_old =x[sub_dim]
        theta_dash = np.where(abs(new_theta - theta_cap)>1)[0]
        if len(theta_dash)>0:
            obj_val = np.inf
            cur_val = x_new
            obj_val = (cost_functn + (np.dot(x_new, np.dot(A_cap, x_new))-
                       2*np.dot(x_new, term1_cap))
                        + g* abs(x_new).sum())
                        
        
            for i in theta_dash:
                x0 = x_new[i]
                y0 = x_old[i]
                p = y0/(y0-x0)
                curr = x_old - p *(x_old - x_new)
                cost = cost_functn + (np.dot(curr, np.dot(A_cap, curr))
                        - 2* np.dot(curr, term1_cap)
                        + g * abs(curr).sum())
                
                if cost < obj_val:
                    obj_val = cost
                    p_new =p
                    cur_val = curr
        else:
            cur_val = x_new
        x[sub_dim] = cur_val
        zeros = sub_dim[np.abs(x[sub_dim]) < tol]
        x[zeros] =0
        theta[sub_dim] = np.int8(np.sign(x[sub_dim]))
        active_set.difference_update(zeros)
        grad = - 2 * term1 + 2 * np.dot(temp, x)
        z_opt = np.max(abs(grad[theta==0]))
        opt_soln = np.max(abs(grad[theta !=0] + g * theta[theta !=0]))
    return x
        
A = []
for i in range(10):
    A.append(np.random.rand(10))
mat = np.array(A)

Y = np.empty([0,1])
for i in range(10):
    Y = np.append(Y, np.random.rand(1))
y = np.array(Y)

gamma=0.2
x=np.zeros(10)
final_sol=feature_sign(mat,Y,gamma)
print(final_sol)
#feature_sign_search(mat,y,gamma,x)
        

[ 0.12162188  0.55706878  0.39593158  0.         -0.33490129  0.04849672
  0.          0.          0.          0.        ]
